# Python Dataclass

<!-- 파이썬에서는 동적타이핑언어로 데이터 타입에 대한 선언이 없어도 충분히 사용이 가능하다. 하지만 프로그램이 확장되고 데이터가 많아질 수록 타입에 대한 정의가 없다면 가독성이 떨어지고 개발자간 의사소통이 어려워진다. 결국 다수의 버그를 발생시키는 상황을 초래할 수 있다. -->

> Dataclass에 정말 상세하고 쉽게 설명해주신 [달레](https://www.daleseo.com/python-dataclasses/)님의 블로그를 참고하여 코드를 각색하였습니다.

## Dataclass 이전의 데이터 정의

In [5]:
from datetime import date

class Car:
    def __init__(
        self, id: int, model: str, create_date: date
    ) -> None:
        self.id = id
        self.model = model
        self.create_date = create_date

k5 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5

dataclass 이전의 데이터를 정의하기 위한 클래스는 `__init__()` 메서드로 인스턴스 변수를 생성시키는데, `__init__()` 메서드에 정의를 하는 것에서 입력, 정의 등에서 여러번 같은 변수명을 입력해야하는 번거로움이 있다.

그리고 인스턴스를 생성하였을 때에도 인스턴스 객체만을 출력하면 생성된 메모리의 위치를 알려줄 뿐이고, 실제 데이터를 감추고 있다.

이때 `__repr__()`메서드를 추가하면 인스턴스를 출력할 때, 필드값이 함께 출력되도록 정의할 수 있다.

In [20]:
from datetime import date

class Car:
    def __init__(
        self, id: int, model: str, create_date: date
    ) -> None:
        self.id = id
        self.model = model
        self.create_date = create_date

    def __repr__(self):
            return (
                self.__class__.__qualname__ + f"(id={self.id!r}, model={self.model!r}, "
                f"create_date={self.create_date!r})"
            )

k5 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5

Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))

In [21]:
# __repr__() !?

다음은 `__repr__()` 구현한 클래스에서 인스턴스를 생성할 때, 동일한 필드를 입력했을 때 동등한지 보겠다.

In [22]:
k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5_2 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
print(k5_1, k5_2, sep="\n")
print(k5_1 == k5_2)

Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
False


동일한 필드가 나오지만, 서로 다른 인스턴스로 취급되는 것을 확인할 수 있다.

이때 동일한 필드를 입력할 때 동일한 인스턴스로 취급하려면, `__eq__()` 메서드를 추가해야한다.

In [44]:
from datetime import date

class Car:
    def __init__(
        self, id: int, model: str, create_date: date
    ) -> None:
        self.id = id
        self.model = model
        self.create_date = create_date

    def __repr__(self):
            return (
                self.__class__.__qualname__ + f"(id={self.id!r}, model={self.model!r}, "
                f"create_date={self.create_date!r})"
            )
# __eq__ 도 나중에
    def __eq__(self, other):
        if other.__class__ is self.__class__:
            return (self.id, self.model, self.create_date) == (
                other.id,
                other.model,
                other.create_date,
            )
        return NotImplemented

In [45]:
k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5_2 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
print(k5_1, k5_2, sep="\n")
print(k5_1 == k5_2)

Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
True


## 데이터 클래스로 작성하기

다음은 위에서 작업하였던 코드들을 dataclass로 구현한다면 어떻게 될지 확인해보도록 하겠다.

In [47]:
from dataclasses import dataclass
from datetime import date


@dataclass
class Car:
    id: int
    model: str
    create_date: date

In [49]:
k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5_2 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
print(k5_1, k5_2, sep="\n")
print(k5_1 == k5_2)

Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17))
True


위에서 구현했던 `__repr__()`, `__eq__()` 등이 dataclass로 랩핑해줌으로써 모두 구현이 되었다. 매우 간단하게 작성이 되었다.

### dataclass frozen 옵션?

위에서 작성된 데이터클래스를 immutable 해지기 위해 frozen 옵션을 사용할 수 있다.

기본 디포트로 False로 세팅되어 있기 때문에 `@dataclass(frozen=True)`와 같이 사용할 수 있다. 

frozen 옵션 `True`와 `False`로 비교해보자.

In [55]:
from dataclasses import dataclass
from datetime import date


@dataclass(frozen=False)
class Car:
    id: int
    model: str
    create_date: date


@dataclass(frozen=True)
class CarFrozen:
    id: int
    model: str
    create_date: date


In [56]:
k5_1 = Car(id=1, model="K5", create_date=date(2022, 2, 17))
k5_1_frozen = CarFrozen(id=1, model="K5", create_date=date(2022, 2, 17))
k5_1, k5_1_frozen

(Car(id=1, model='K5', create_date=datetime.date(2022, 2, 17)),
 CarFrozen(id=1, model='K5', create_date=datetime.date(2022, 2, 17)))

In [57]:
k5_1.model = "K3"
k5_1

Car(id=1, model='K3', create_date=datetime.date(2022, 2, 17))

k5_1의 model을 K3로 변경해 사용하였다.

In [58]:
k5_1_frozen.model = "K3"
k5_1_frozen

FrozenInstanceError: cannot assign to field 'model'

위와 같이 Frozen True일때 에러가 발생하게 된다.